
# StockAlert · Fast Prompting POC

**Objetivo:** Mostrar cómo técnicas de *Fast Prompting* mejoran una tarea de sugerir pedidos y alertas de stock en un lote de SKUs, reduciendo costo (menos llamadas) y aumentando consistencia (JSON válido, reglas claras).

> La notebook corre 100% offline con un **Mock LLM**.
> Si querés integrar un proveedor real, hay una celda para habilitarlo.


In [ ]:

# Imports
import os, json, math, hashlib, textwrap, random
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta

from pathlib import Path
DATA = Path("../data")
SRC = Path("../src")

# Visual settings (no custom colors per project rules)
plt.rcParams['figure.figsize'] = (8, 4)

print("Ready. Python version OK.")


In [ ]:

# Load data
products = pd.read_csv(DATA / "products.csv")
sales = pd.read_csv(DATA / "sales.csv", parse_dates=["date"])

# Compute a simple 7-day avg sales (synthetic)
recent = sales[sales['date'] >= sales['date'].max() - pd.Timedelta(days=7)]
avg_sales = recent.groupby('product_id')['units_sold'].mean().rename('avg_7d')
df = products.merge(avg_sales, on='product_id', how='left').fillna({'avg_7d': 0})

df



## Prompt variants

- **Baseline**: prompt largo sin estructura rígida.
- **Fast-01**: rol+tarea + **JSON schema** + campos obligatorios + manejo de faltantes.
- **Fast-02**: agrega **few-shot** y reglas explícitas, manteniendo salida estricta.


In [ ]:

# Load prompt templates
import importlib.util, sys
spec = importlib.util.spec_from_file_location("pipeline", SRC / "pipeline.py")
pipeline = importlib.util.module_from_spec(spec)
sys.modules["pipeline"] = pipeline
spec.loader.exec_module(pipeline)

baseline = pipeline.baseline_template()
fast01 = pipeline.fast01_template()
fast02 = pipeline.fast02_template()

baseline, fast01, fast02



## Mock LLM (offline)

Para esta POC, usamos un **simulador** que aplica reglas heurísticas y **finge** la salida del LLM en **JSON**.
Esto nos permite medir:
- **Validez de formateo** (siempre válido en el mock, pero contabilizamos como proxy).
- **Acierto** de *REORDER vs HOLD* comparado con una heurística de referencia.
- **Costo estimado**: nº de llamadas simuladas y longitud de prompt.


In [ ]:

def simple_heuristic(row):
    # Reference rule for evaluation (ground truth proxy)
    # Reorder if stock below reorder_point; if avg_7d very low and shelf life short, prefer HOLD
    if row['current_stock'] < row['reorder_point']:
        risk_expiry = 'LOW'
        if row['shelf_life_days'] <= 20 and row['avg_7d'] < 10:
            risk_expiry = 'HIGH'
        return {
            "product_id": row['product_id'],
            "action": "REORDER" if risk_expiry != 'HIGH' else "HOLD",
            "qty": int(row['reorder_qty'] if risk_expiry != 'HIGH' else 0),
            "reasons": ["stock_bajo"] if risk_expiry != 'HIGH' else ["riesgo_vencimiento"],
            "risk_expiry": risk_expiry
        }
    else:
        return {
            "product_id": row['product_id'],
            "action": "HOLD",
            "qty": 0,
            "reasons": ["stock_suficiente"],
            "risk_expiry": "LOW"
        }

def mock_llm(prompt_variant, payload_products):
    # Produce a JSON compatible list using a similar (but not identical) rule to create noise
    results = []
    for row in payload_products.to_dict(orient='records'):
        # Small variation vs heuristic to simulate "model" differences
        base = simple_heuristic(pd.Series(row))
        # tweak: sometimes suggest smaller qty when close to reorder_point
        if base["action"] == "REORDER" and row['current_stock'] >= (row['reorder_point'] * 0.9):
            base["qty"] = max(0, base["qty"] - int(row['reorder_qty']*0.2))
            base["reasons"].append("ajuste_fino")
        results.append(base)
    return {"results": results}

def run_batch(prompt_variant, batch_df, max_skus_per_call=6):
    # Consolidate multiple SKUs in one "call" to reduce cost
    calls = []
    results = []
    for i in range(0, len(batch_df), max_skus_per_call):
        sub = batch_df.iloc[i:i+max_skus_per_call]
        payload = {"products": sub.to_dict(orient='records')}
        # Simulate token size as the length of the JSON payload + template header size
        prompt_len = len(json.dumps(payload)) + len(prompt_variant.template)
        response = mock_llm(prompt_variant, sub)
        calls.append({"prompt_len": prompt_len, "n_skus": len(sub)})
        results.extend(response["results"])
    return results, calls

def evaluate(results, reference_df):
    # Compare REORDER vs HOLD decision against heuristic
    ref = []
    for _, row in reference_df.iterrows():
        gt = simple_heuristic(row)  # ground truth proxy
        ref.append((row['product_id'], gt['action']))
    ref_map = dict(ref)
    acc = 0
    for r in results:
        if ref_map.get(r['product_id']) == r['action']:
            acc += 1
    return acc / len(reference_df)

df_eval = df.copy()
df_eval.head()



## Experimentos

Comparamos **Baseline**, **Fast-01** y **Fast-02**, con *batching* de 6 SKUs por llamada.
Métricas:
- **Accuracy** vs. heurística (proxy).
- **Prompts por SKU** (costo): nº de llamadas y tamaño promedio del prompt.


In [ ]:

variants = [baseline, fast01, fast02]

records = []
for v in variants:
    res, calls = run_batch(v, df_eval, max_skus_per_call=6)
    acc = evaluate(res, df_eval)
    total_calls = len(calls)
    avg_prompt_len = sum(c['prompt_len'] for c in calls)/max(1, total_calls)
    records.append({
        "variant": v.name,
        "accuracy_proxy": round(acc, 3),
        "n_calls": total_calls,
        "avg_prompt_len_chars": int(avg_prompt_len)
    })

exp = pd.DataFrame(records)
exp


In [ ]:

# Simple bar chart for the accuracy proxy
plt.figure()
plt.bar(exp['variant'], exp['accuracy_proxy'])
plt.title("Accuracy proxy por variante")
plt.xlabel("Variante")
plt.ylabel("Accuracy proxy (0-1)")
plt.show()



## Resultados y análisis

- **Fast-01** y **Fast-02** utilizan **salida JSON estricta** → evita rondas extra para corregir formato.
- **Few-shot** (Fast-02) y reglas explícitas mejoran la consistencia con las heurísticas.
- **Costo**: con *batching* (6 SKUs / llamada) reducimos a 1 llamada para el dataset ejemplo.

> En un entorno real, además mediríamos tokens (prompt+completion) y latencia.



## (Opcional) Integración con proveedor LLM

Descomenta y completa con tu API key para probar en producción. Mantén el JSON schema y el batching para controlar costos.

**Nota:** Esta celda está comentada para que la notebook sea 100% ejecutable sin internet ni credenciales.


In [ ]:

# %%
# import os, json
# from openai import OpenAI
# client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
#
# def real_llm_call(prompt_text: str) -> dict:
#     # Ajusta el modelo según tu acceso (ej: 'gpt-4o-mini' o similar)
#     resp = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {"role":"system", "content":"Eres un analista de inventario experto. Responde SOLO con JSON válido."},
#             {"role":"user", "content": prompt_text},
#         ],
#         temperature=0.1,
#         max_tokens=600
#     )
#     txt = resp.choices[0].message.content
#     return json.loads(txt)
#
# # Para usar: reemplazar mock_llm(...) por real_llm_call(...)



## Conclusión (vs. Preentrega 1)

- Pasamos de una idea **descriptiva/marketing** a una **POC reproducible**.
- Las técnicas de **Fast Prompting** (schema, few-shot, constraints, batching) mejoran **calidad y costos**.
- El siguiente paso es integrar el LLM real y conectar con tu fuente de datos/ERP para pruebas con productos reales.
